In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False
==== Kiro Tool connected to /dev/ttyUSB0 (115200) ====
[KTOOL] initialize
[KTOOL] enable
[KTOOL] op_init
[KTOOL] led_off


### Make scene

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -3, 3, -3, 3, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [4]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
gscene.show_pose(crob.home_pose)

HOLD_NAME = "holder"
front_dist = 0.45+CLEARANCE
holder_g = gscene.create_safe(GEOTYPE.SPHERE, HOLD_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, BAG_DIM[2]/2), rpy=(0,np.pi/2,0),
                           fixed=False, collision=False, color=(1,0,0,0.5))
holder_s = pscene.create_binder(HOLD_NAME, HOLD_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=BAG_KEY)
floor_s = pscene.create_binder("floor_ws", "floor_ws", PlacePlane, point=None)

goal_g = gscene.create_safe(GEOTYPE.BOX, "goal", link_name="base_link",dims=(0.4,0.4,WALL_THICKNESS),
                           center=(2, 0, -WALL_THICKNESS/2-CLEARANCE), rpy=(0,0,0),
                           fixed=False, collision=False, color=(1,0,0,0.5))
goal_s = pscene.create_binder("goal", "goal", PlacePlane, point=None)

## Bag

In [5]:
BAG_NAME = "bag"
BAG_DIM = (0.4, 0.29,0.635)

In [6]:
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [7]:
bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME, link_name="base_link",dims=BAG_DIM,
                           center=(1,0,BAG_DIM[2]/2), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.8,0.8,1))
handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                           center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME)

BAG_KEY = 1
action_points_dict = {}
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
    ap_name = "side{}".format(i)
    action_points_dict[ap_name]=FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=BAG_KEY)

action_points_dict["bottom_p"] = PlacePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0))

bag_s = pscene.create_subject(oname=BAG_NAME, gname=BAG_NAME, _type=CustomObject, action_points_dict=action_points_dict)

### prepare planner

In [11]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
generate table - Handlegenerate table - Geometry

generate table - Binder
generate table - Object


In [20]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=30)
tplan.prepare()

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [21]:
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan.motion_filters = [GraspChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [22]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['bag']
('floor_ws',)


In [24]:
goal_nodes = [('goal',)]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

Use 18/36 agents
try: 0 - ('floor_ws',)->('holder',)
error: 0.35
try transition motion
try: 0 - ('floor_ws',)->('holder',)
error: 0.35
try transition motion
transition motion tried: True
result: 0 - ('floor_ws',)->('holder',) = success
branching: 0->1 (0.16/100.0 s, steps/err: 3(83.0528736115 ms)/0.00126296048801)
transition motion tried: True
try: 1 - ('holder',)->('goal',)
result: 0 - ('floor_ws',)->('holder',) = success
error: 1.6
branching: 0->2 (0.18/100.0 s, steps/err: 3(80.4529190063 ms)/0.000901098207919)
try transition motion
try: 2 - ('holder',)->('goal',)
error: 3.1
try transition motion
try: 1 - ('holder',)->('goal',)
error: 2.8
transition motion tried: True
try transition motion
Goal reached
result: 1 - ('holder',)->('goal',) = success
branching: 1->3 (0.26/100.0 s, steps/err: 11(64.7900104523 ms)/0.000446631007035)
transition motion tried: True
Goal reached
++ adding return motion to acquired answer ++
result: 2 - ('holder',)->('goal',) = success
branching: 2->4 (0.29/100

Goal reached
++ adding return motion to acquired answer ++
result: 2 - ('holder',)->('goal',) = success
branching: 2->14 (0.68/100.0 s, steps/err: 8(121.86217308 ms)/0.00163006013124)
++ adding return motion to acquired answer ++
transition motion tried: True
result: 1 - ('holder',)->('floor_ws',) = success
branching: 1->15 (0.7/100.0 s, steps/err: 7(129.878997803 ms)/0.00135023429686)
======================= terminated 0: Stop called from other agent  (0.7/100.0) ===============================
transition motion tried: True
Goal reached
result: 0 - ('floor_ws',)->('holder',) = success
branching: 0->17 (0.71/100.0 s, steps/err: 41(230.810880661 ms)/0.00146706655741)
transition motion tried: True
======================= terminated 9: Stop called from other agent  (0.7/100.0) ===============================
Goal reached
result: 2 - ('holder',)->('floor_ws',) = success
branching: 2->18 (0.73/100.0 s, steps/err: 22(114.536046982 ms)/0.00167670436979)
Goal reached
======================= te

In [25]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [26]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('floor_ws',)->('holder',)
('holder',)->('goal',)
('goal',)->('goal',)
